<a href="https://colab.research.google.com/github/abhishekkm8088/Pyspark-Abhi/blob/main/Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("hello world")


hello world


##LINUX BASIC

In [11]:
!cat /etc/os-release
!uname -a


PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy
Linux ff3c83f75672 6.1.123+ #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux


In [13]:
!pwd


/content


In [14]:
!whoami

root


##Pyspark Basics and dataframes

In [17]:
pip install pyspark


In [19]:
from pyspark.sql import SparkSession

In [21]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [26]:
#create Dataframes
data =[("hello", "world")]
columns = ["word1", "word2"]

df = spark.createDataFrame(data, columns)

In [28]:
df.show()

+-----+-----+
|word1|word2|
+-----+-----+
|hello|world|
+-----+-----+



##Basic Transformation and action

In [30]:
columns = ["name", "department", "salary"]
data = [
    ("John", "Sales", 3000),
    ("Jane", "Finance", 4000),
    ("Mike", "Sales", 3500),
    ("Alice", "Finance", 3800),
    ("Bob", "IT", 4500)
]
df = spark.createDataFrame(data, columns)


In [32]:
df.show()

+-----+----------+------+
| name|department|salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [34]:
#filter: employees with salary > 3500
df_filtered = df.filter(df.salary > 3500)
df_filtered.show()

+-----+----------+------+
| name|department|salary|
+-----+----------+------+
| Jane|   Finance|  4000|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [36]:
#GroupBY and Aggregate: Average salary by department
df_agg = df.groupBy("department").agg({"salary": "avg"})
df_agg.show()

+----------+-----------+
|department|avg(salary)|
+----------+-----------+
|     Sales|     3250.0|
|   Finance|     3900.0|
|        IT|     4500.0|
+----------+-----------+



In [40]:
# Add new column : salary with bonos (10%)
from pyspark.sql.functions import col
exp=col("salary") * 1.1
df_with_bonus = df.withColumn("salary_with_bonus", exp)
df_with_bonus.show()

+-----+----------+------+------------------+
| name|department|salary| salary_with_bonus|
+-----+----------+------+------------------+
| John|     Sales|  3000|3300.0000000000005|
| Jane|   Finance|  4000|            4400.0|
| Mike|     Sales|  3500|3850.0000000000005|
|Alice|   Finance|  3800|            4180.0|
|  Bob|        IT|  4500|            4950.0|
+-----+----------+------+------------------+



In [42]:
from pyspark.sql.functions import col,upper,lower,concat_ws,when

df.show()

+-----+----------+------+
| name|department|salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [44]:
#Change case Transformation
df_upper = df.withColumn("name_upper", upper(col("name")))
df.lower=df.withColumn("name_lower", lower(col("name")))
df_upper.show()
df.lower.show()

+-----+----------+------+----------+
| name|department|salary|name_upper|
+-----+----------+------+----------+
| John|     Sales|  3000|      JOHN|
| Jane|   Finance|  4000|      JANE|
| Mike|     Sales|  3500|      MIKE|
|Alice|   Finance|  3800|     ALICE|
|  Bob|        IT|  4500|       BOB|
+-----+----------+------+----------+

+-----+----------+------+----------+
| name|department|salary|name_lower|
+-----+----------+------+----------+
| John|     Sales|  3000|      john|
| Jane|   Finance|  4000|      jane|
| Mike|     Sales|  3500|      mike|
|Alice|   Finance|  3800|     alice|
|  Bob|        IT|  4500|       bob|
+-----+----------+------+----------+



In [46]:
#concatenate columns
df_concat = df.withColumn("name_department", concat_ws(" - ", col("name"), col("department")))
df_concat.show()

+-----+----------+------+---------------+
| name|department|salary|name_department|
+-----+----------+------+---------------+
| John|     Sales|  3000|   John - Sales|
| Jane|   Finance|  4000| Jane - Finance|
| Mike|     Sales|  3500|   Mike - Sales|
|Alice|   Finance|  3800|Alice - Finance|
|  Bob|        IT|  4500|       Bob - IT|
+-----+----------+------+---------------+



In [48]:
#string length of names in new DF
from pyspark.sql.functions import length
df_length = df.withColumn("name_length", length(col("name")))
df_length.show()

+-----+----------+------+-----------+
| name|department|salary|name_length|
+-----+----------+------+-----------+
| John|     Sales|  3000|          4|
| Jane|   Finance|  4000|          4|
| Mike|     Sales|  3500|          4|
|Alice|   Finance|  3800|          5|
|  Bob|        IT|  4500|          3|
+-----+----------+------+-----------+



In [51]:
#add some conditional columns (salary, category) high medium and low
df_with_category = df.withColumn(
    "category",
    when(col("salary") >= 4000, "High")
    .when((col("salary") >= 3500) & (col("salary") < 4000), "Medium")
    .otherwise("Low")
)
df_with_category.show()




+-----+----------+------+--------+
| name|department|salary|category|
+-----+----------+------+--------+
| John|     Sales|  3000|     Low|
| Jane|   Finance|  4000|    High|
| Mike|     Sales|  3500|  Medium|
|Alice|   Finance|  3800|  Medium|
|  Bob|        IT|  4500|    High|
+-----+----------+------+--------+



In [1]:
# Rename Column (Salary --> Base_salary)
df_renamed = df.withColumnRenamed("salary", "base_salary")
df_renamed.show()

NameError: name 'df' is not defined

In [3]:
from pyspark.sql import SparkSession
spark =SparkSession.builder.appName("Basics").getOrCreate()
columns = ["Name","Department","Salary"]
data = [
    ("John", "Sales", 3000),
    ("Jane", "Finance", 4000),
    ("Mike", "Sales", 3500),
    ("Alice", "Finance", 3800),
    ("Bob", "IT", 4500)
]

df = spark.createDataFrame(data, columns)

df.show()

+-----+----------+------+
| Name|Department|Salary|
+-----+----------+------+
| John|     Sales|  3000|
| Jane|   Finance|  4000|
| Mike|     Sales|  3500|
|Alice|   Finance|  3800|
|  Bob|        IT|  4500|
+-----+----------+------+



In [6]:
#Group by department and count employees
df_count = df.groupBy("Department").agg({"Name": "count"})
df_count.show()

+----------+-----------+
|Department|count(Name)|
+----------+-----------+
|     Sales|          2|
|   Finance|          2|
|        IT|          1|
+----------+-----------+



In [8]:
#group by Department and calcualte avarge salary
df.groupBy("Department").avg("salary").show()


+----------+-----------+
|Department|avg(salary)|
+----------+-----------+
|     Sales|     3250.0|
|   Finance|     3900.0|
|        IT|     4500.0|
+----------+-----------+



In [10]:
#Group by departemnt and calculate multiple aggregation:
df.groupBy("Department").agg({"Salary": "avg", "Salary": "max", "Salary": "min"}).show()

+----------+-----------+
|Department|min(Salary)|
+----------+-----------+
|     Sales|       3000|
|   Finance|       3800|
|        IT|       4500|
+----------+-----------+



In [13]:
from pyspark.sql import functions as F
df.groupBy("Department").agg(F.avg("Salary"), F.max("Salary"), F.min("Salary")).show()

+----------+-----------+-----------+-----------+
|Department|avg(Salary)|max(Salary)|min(Salary)|
+----------+-----------+-----------+-----------+
|     Sales|     3250.0|       3500|       3000|
|   Finance|     3900.0|       4000|       3800|
|        IT|     4500.0|       4500|       4500|
+----------+-----------+-----------+-----------+



In [14]:
# Create another DataFrame for department info
dept_data = [
    ("Sales", "Building A"),
    ("Finance", "Building B"),
    ("IT", "Building C")
]
dept_columns = ["Department", "Location"]

In [19]:
dept_df = spark.createDataFrame(dept_data, dept_columns)
#join employee data with department info
df_joined = df.join(dept_df, on="Department", how="inner")
df_joined.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|   Finance| Jane|  4000|Building B|
|   Finance|Alice|  3800|Building B|
|        IT|  Bob|  4500|Building C|
|     Sales| John|  3000|Building A|
|     Sales| Mike|  3500|Building A|
+----------+-----+------+----------+



In [20]:
dept_df.show()

+----------+----------+
|Department|  Location|
+----------+----------+
|     Sales|Building A|
|   Finance|Building B|
|        IT|Building C|
+----------+----------+



In [22]:
dept_df = spark.createDataFrame(dept_data, dept_columns)
#join employee data with department info
df_joined = df.join(dept_df, on="Department", how="left")
df_joined.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|     Sales| John|  3000|Building A|
|   Finance| Jane|  4000|Building B|
|     Sales| Mike|  3500|Building A|
|   Finance|Alice|  3800|Building B|
|        IT|  Bob|  4500|Building C|
+----------+-----+------+----------+



In [24]:
#perform inner join
df_joined = df.join(dept_df, on="Department", how="inner")
df_joined.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|   Finance| Jane|  4000|Building B|
|   Finance|Alice|  3800|Building B|
|        IT|  Bob|  4500|Building C|
|     Sales| John|  3000|Building A|
|     Sales| Mike|  3500|Building A|
+----------+-----+------+----------+



In [26]:
#perform left join employee, dept info if exist
df_joined = df.join(dept_df, on="Department", how="left")
df_joined.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|     Sales| John|  3000|Building A|
|   Finance| Jane|  4000|Building B|
|     Sales| Mike|  3500|Building A|
|   Finance|Alice|  3800|Building B|
|        IT|  Bob|  4500|Building C|
+----------+-----+------+----------+



In [28]:
#perform right join employee, dept info if exist
df_joined = df.join(dept_df, on="Department", how="right")
df_joined.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|     Sales| Mike|  3500|Building A|
|     Sales| John|  3000|Building A|
|   Finance|Alice|  3800|Building B|
|   Finance| Jane|  4000|Building B|
|        IT|  Bob|  4500|Building C|
+----------+-----+------+----------+



In [30]:
#right outer join (all department, emp info if exist )
df_joined = df.join(dept_df, on="Department", how="rightouter")
df_joined.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|     Sales| Mike|  3500|Building A|
|     Sales| John|  3000|Building A|
|   Finance|Alice|  3800|Building B|
|   Finance| Jane|  4000|Building B|
|        IT|  Bob|  4500|Building C|
+----------+-----+------+----------+



In [31]:
#full outer join (all department, emp info if exist )
df_joined = df.join(dept_df, on="Department", how="fullouter")
df_joined.show()

+----------+-----+------+----------+
|Department| Name|Salary|  Location|
+----------+-----+------+----------+
|   Finance| Jane|  4000|Building B|
|   Finance|Alice|  3800|Building B|
|        IT|  Bob|  4500|Building C|
|     Sales| John|  3000|Building A|
|     Sales| Mike|  3500|Building A|
+----------+-----+------+----------+



##Sample Datasets: Project


In [33]:
  from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, sum


In [35]:
# Start Spark session
spark = SparkSession.builder.appName("RevenueCalculation").getOrCreate()


In [37]:
# Step 2: Create sales_data DataFrame
sales_data = [
    ("1", "P101", 2, 500),
    ("2", "P102", 1, 300),
    ("3", "P101", 1, 500)
]
sales_df = spark.createDataFrame(sales_data, ["sale_id", "product_id", "quantity", "price"])

print("🔹 Step 2: Sales Data")
sales_df.show()


🔹 Step 2: Sales Data
+-------+----------+--------+-----+
|sale_id|product_id|quantity|price|
+-------+----------+--------+-----+
|      1|      P101|       2|  500|
|      2|      P102|       1|  300|
|      3|      P101|       1|  500|
+-------+----------+--------+-----+



In [39]:
# Step 3: Create product_data DataFrame
product_data = [
    ("P101", "Mobile Phone"),
    ("P102", "Power Bank")
]
product_df = spark.createDataFrame(product_data, ["product_id", "product_name"])

print("🔹 Step 3: Product Data")
product_df.show()


🔹 Step 3: Product Data
+----------+------------+
|product_id|product_name|
+----------+------------+
|      P101|Mobile Phone|
|      P102|  Power Bank|
+----------+------------+



In [41]:
# Step 4: Add revenue column to sales_df
sales_df = sales_df.withColumn("revenue", col("quantity") * col("price"))

print("🔹 Step 4: Sales Data with Revenue")
sales_df.show()

🔹 Step 4: Sales Data with Revenue
+-------+----------+--------+-----+-------+
|sale_id|product_id|quantity|price|revenue|
+-------+----------+--------+-----+-------+
|      1|      P101|       2|  500|   1000|
|      2|      P102|       1|  300|    300|
|      3|      P101|       1|  500|    500|
+-------+----------+--------+-----+-------+



In [43]:
# Step 5: Join sales_df with product_df
joined_df = sales_df.join(product_df, "product_id")

print("🔹 Step 5: Joined DataFrame")
joined_df.show()

🔹 Step 5: Joined DataFrame
+----------+-------+--------+-----+-------+------------+
|product_id|sale_id|quantity|price|revenue|product_name|
+----------+-------+--------+-----+-------+------------+
|      P101|      1|       2|  500|   1000|Mobile Phone|
|      P101|      3|       1|  500|    500|Mobile Phone|
|      P102|      2|       1|  300|    300|  Power Bank|
+----------+-------+--------+-----+-------+------------+



In [45]:
# Step 6: Group by product and calculate total revenue
revenue_df = joined_df.groupBy("product_id", "product_name").agg(sum("revenue").alias("total_revenue"))

print("🔹 Step 6: Final Revenue by Product")
revenue_df.show()

🔹 Step 6: Final Revenue by Product
+----------+------------+-------------+
|product_id|product_name|total_revenue|
+----------+------------+-------------+
|      P101|Mobile Phone|         1500|
|      P102|  Power Bank|          300|
+----------+------------+-------------+



In [48]:
#data loading in dataframe-sales_data
sf=spark.read.csv("/content/sales_data.csv", header=True, inferSchema=True)
sf.show()


+-------+----------+-----------+-------------------+--------+-----+
|sale_id|product_id|customer_id|          sale_date|quantity|price|
+-------+----------+-----------+-------------------+--------+-----+
|   1001|       101|        501|2025-07-10 08:23:00|       3| 25.5|
|   1002|       102|        502|2025-07-11 09:45:00|       2| 15.0|
|   1003|       103|        503|2025-07-12 10:15:00|       1| 30.0|
|   1004|       101|        504|2025-07-13 12:20:00|       5| 25.5|
|   1005|       105|        505|2025-07-14 14:35:00|      10| 45.0|
|   1006|       102|        506|2025-07-15 16:00:00|       4| 15.0|
|   1007|       106|        507|2025-07-16 17:10:00|       3| 40.0|
|   1008|       107|        508|2025-07-17 18:25:00|       2| 60.0|
|   1009|       108|        509|2025-07-18 19:30:00|       7| 25.0|
|   1010|       109|        510|2025-07-19 20:45:00|       6| 50.0|
|   1011|       110|        511|2025-07-20 21:55:00|       4| 12.5|
|   1012|       111|        512|2025-07-21 22:30

In [50]:
#data loading in dataframe-customer_data
sf=spark.read.csv("/content/customer_data.csv", header=True, inferSchema=True)
sf.show()

+-----------+-------------+-------------------+-------------------+
|customer_id|customer_name|              email|          join_date|
+-----------+-------------+-------------------+-------------------+
|        501|        Alice|  alice@example.com|2025-05-20 10:10:00|
|        502|          Bob|    bob@example.com|2025-06-15 14:00:00|
|        503|      Charlie|charlie@example.com|2025-04-05 09:50:00|
|        504|        David|  david@example.com|2025-07-01 12:25:00|
|        505|         Emma|   emma@example.com|2025-07-10 15:30:00|
|        506|        Frank|  frank@example.com|2025-03-23 17:00:00|
|        507|        Grace|  grace@example.com|2025-05-01 13:20:00|
|        508|        Henry|  henry@example.com|2025-06-07 10:10:00|
|        509|       Isabel| isabel@example.com|2025-05-25 16:30:00|
|        510|         Jack|   jack@example.com|2025-04-12 11:55:00|
|        511|         Kate|   kate@example.com|2025-06-18 14:10:00|
|        512|         Liam|   liam@example.com|2

In [53]:
#data loading in dataframe-pruduct_data
sf=spark.read.csv("/content/product_data.csv", header=True, inferSchema=True)
sf.show()

+----------+------------+-------------+
|product_id|product_name|     category|
+----------+------------+-------------+
|       101|    Widget A|      Gadgets|
|       102|    Widget B|      Gadgets|
|       103|    Widget C|  Electronics|
|       104|    Widget D|  Electronics|
|       105|    Widget E|Home & Living|
|       106|    Widget F|Home & Living|
|       107|    Widget G|    Furniture|
|       108|    Widget H|      Gadgets|
|       109|    Widget I|    Furniture|
|       110|    Widget J|  Electronics|
|       111|    Widget K|Home & Living|
|       112|    Widget L|  Electronics|
|       113|    Widget M|      Gadgets|
|       114|    Widget N|    Furniture|
|       115|    Widget O|      Gadgets|
|       116|    Widget P|Home & Living|
|       117|    Widget Q|  Electronics|
|       118|    Widget R|    Furniture|
|       119|    Widget S|      Gadgets|
|       120|    Widget T|    Furniture|
+----------+------------+-------------+
only showing top 20 rows



In [55]:
# Step 1: Start a SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RetailSalesAnalysis") \
    .getOrCreate()






In [57]:
# Step 2: Read the CSV files into DataFrames
# Load sales data
sales_df = spark.read.csv("sales_data.csv", header=True, inferSchema=True)

# Load product data
product_df = spark.read.csv("product_data.csv", header=True, inferSchema=True)

# Load customer data
customer_df = spark.read.csv("customer_data.csv", header=True, inferSchema=True)

In [59]:
#Step3: Verify DataFrames using .show()
sales_df.show()
product_df.show()
customer_df.show()


+-------+----------+-----------+-------------------+--------+-----+
|sale_id|product_id|customer_id|          sale_date|quantity|price|
+-------+----------+-----------+-------------------+--------+-----+
|   1001|       101|        501|2025-07-10 08:23:00|       3| 25.5|
|   1002|       102|        502|2025-07-11 09:45:00|       2| 15.0|
|   1003|       103|        503|2025-07-12 10:15:00|       1| 30.0|
|   1004|       101|        504|2025-07-13 12:20:00|       5| 25.5|
|   1005|       105|        505|2025-07-14 14:35:00|      10| 45.0|
|   1006|       102|        506|2025-07-15 16:00:00|       4| 15.0|
|   1007|       106|        507|2025-07-16 17:10:00|       3| 40.0|
|   1008|       107|        508|2025-07-17 18:25:00|       2| 60.0|
|   1009|       108|        509|2025-07-18 19:30:00|       7| 25.0|
|   1010|       109|        510|2025-07-19 20:45:00|       6| 50.0|
|   1011|       110|        511|2025-07-20 21:55:00|       4| 12.5|
|   1012|       111|        512|2025-07-21 22:30

In [61]:
#Step 4: Create Temporary Views (Optional)
sales_df.createOrReplaceTempView("sales_data")
product_df.createOrReplaceTempView("product_data")
customer_df.createOrReplaceTempView("customer_data")


In [63]:
#Step 5: Solve Problem 1.1 – Total Revenue Per Product
from pyspark.sql.functions import col, sum as _sum

sales_with_revenue = sales_df.withColumn("revenue", col("quantity") * col("price"))

total_revenue_per_product = sales_with_revenue.groupBy("product_id") \
    .agg(_sum("revenue").alias("total_revenue"))

total_revenue_per_product.show()


+----------+-------------+
|product_id|total_revenue|
+----------+-------------+
|       148|         58.0|
|       137|        115.0|
|       133|        115.5|
|       108|        235.0|
|       101|        280.5|
|       115|        484.0|
|       126|        240.0|
|       103|        240.0|
|       128|         90.0|
|       122|        100.0|
|       111|        292.5|
|       140|         80.0|
|       132|        132.0|
|       146|        180.0|
|       142|        190.0|
|       139|         55.5|
|       120|        211.5|
|       117|        114.0|
|       112|        231.0|
|       127|         70.0|
+----------+-------------+
only showing top 20 rows



In [65]:
#Started 1 aggregation
#We walked through Total Revenue Per Product, calculating:
from pyspark.sql.functions import col, sum as _sum

sales_with_revenue = sales_df.withColumn("revenue", col("quantity") * col("price"))

total_revenue_per_product = sales_with_revenue.groupBy("product_id") \
    .agg(_sum("revenue").alias("total_revenue"))

total_revenue_per_product.show()


+----------+-------------+
|product_id|total_revenue|
+----------+-------------+
|       148|         58.0|
|       137|        115.0|
|       133|        115.5|
|       108|        235.0|
|       101|        280.5|
|       115|        484.0|
|       126|        240.0|
|       103|        240.0|
|       128|         90.0|
|       122|        100.0|
|       111|        292.5|
|       140|         80.0|
|       132|        132.0|
|       146|        180.0|
|       142|        190.0|
|       139|         55.5|
|       120|        211.5|
|       117|        114.0|
|       112|        231.0|
|       127|         70.0|
+----------+-------------+
only showing top 20 rows



In [67]:
#Step 2: Total Quantity Sold Per Customer
#We’ll calculate how much quantity each customer has purchased in total.
from pyspark.sql.functions import sum as _sum

# Group sales by customer_id and sum quantity
total_quantity_per_customer = sales_df.groupBy("customer_id") \
    .agg(_sum("quantity").alias("total_quantity"))

# Show result
total_quantity_per_customer.show()


+-----------+--------------+
|customer_id|total_quantity|
+-----------+--------------+
|        540|             3|
|        516|             4|
|        513|             8|
|        530|             2|
|        501|             3|
|        548|             9|
|        539|             6|
|        519|             6|
|        556|             8|
|        512|             9|
|        570|             3|
|        559|             6|
|        545|             3|
|        507|             3|
|        544|             7|
|        542|             8|
|        553|             7|
|        560|             5|
|        504|             5|
|        554|             5|
+-----------+--------------+
only showing top 20 rows



In [69]:
#Step 3: Average Revenue Per Customer
#This shows how much revenue each customer generates on average per transaction.
from pyspark.sql.functions import sum as _sum, count, round

# First, calculate total revenue and number of transactions per customer
customer_revenue_stats = sales_df.withColumn("revenue", sales_df["quantity"] * sales_df["price"]) \
    .groupBy("customer_id") \
    .agg(
        _sum("revenue").alias("total_revenue"),
        count("*").alias("num_transactions")
    )

# Then calculate average revenue per transaction
avg_revenue_per_customer = customer_revenue_stats.withColumn(
    "avg_revenue", round(customer_revenue_stats["total_revenue"] / customer_revenue_stats["num_transactions"], 2)
)

# Show result
avg_revenue_per_customer.show()


+-----------+-------------+----------------+-----------+
|customer_id|total_revenue|num_transactions|avg_revenue|
+-----------+-------------+----------------+-----------+
|        540|         55.5|               1|       55.5|
|        516|        128.0|               1|      128.0|
|        513|        180.0|               1|      180.0|
|        530|         76.0|               1|       76.0|
|        501|         76.5|               1|       76.5|
|        548|        198.0|               1|      198.0|
|        539|        210.0|               1|      210.0|
|        519|        111.0|               1|      111.0|
|        556|        400.0|               1|      400.0|
|        512|         90.0|               1|       90.0|
|        570|        120.0|               1|      120.0|
|        559|        108.0|               1|      108.0|
|        545|         91.5|               1|       91.5|
|        507|        120.0|               1|      120.0|
|        544|         84.0|    

In [71]:
#Top 5 Customers by Total Revenue
#This gives the top 5 customers who spent the most overall.
from pyspark.sql.functions import col

top_customers = sales_df.withColumn("revenue", col("quantity") * col("price")) \
    .groupBy("customer_id") \
    .agg(_sum("revenue").alias("total_revenue")) \
    .orderBy(col("total_revenue").desc()) \
    .limit(5)

top_customers.show()


+-----------+-------------+
|customer_id|total_revenue|
+-----------+-------------+
|        505|        450.0|
|        556|        400.0|
|        565|        356.0|
|        510|        300.0|
|        520|        280.0|
+-----------+-------------+



In [73]:
#Step 5: Top 3 Selling Categories by Quantity
#Here we’ll join sales_df with product_df, then group by category and sum quantity.
# Join sales data with product data to get category info
sales_with_category = sales_df.join(product_df, on="product_id", how="inner")

# Group by category and calculate total quantity sold
top_categories = sales_with_category.groupBy("category") \
    .agg(_sum("quantity").alias("total_quantity")) \
    .orderBy(col("total_quantity").desc()) \
    .limit(3)

top_categories.show()



+-------------+--------------+
|     category|total_quantity|
+-------------+--------------+
|      Gadgets|            99|
|Home & Living|            88|
|    Furniture|            81|
+-------------+--------------+



In [75]:
#step6:Find Returning Customers (More Than 1 Purchase)
#We’ll analyze the number of purchases each customer made and filter those with more than one.
# Count number of purchases per customer
returning_customers = sales_df.groupBy("customer_id") \
    .agg(count("sale_id").alias("num_purchases")) \
    .filter(col("num_purchases") > 1)

# Join with customer_df to get their details
returning_customers_info = returning_customers.join(customer_df, on="customer_id", how="inner")

returning_customers_info.select("customer_id", "customer_name", "email", "num_purchases").show()



+-----------+-------------+-----+-------------+
|customer_id|customer_name|email|num_purchases|
+-----------+-------------+-----+-------------+
+-----------+-------------+-----+-------------+



In [77]:
#Step 7: Top 5 Customers by Total Amount Spent
#We'll calculate the total spending by each customer (quantity * price) and sort to get the top 5.
# Calculate total amount per transaction
sales_df = sales_df.withColumn("total_amount", col("quantity") * col("price"))

# Aggregate total spend per customer
customer_spending = sales_df.groupBy("customer_id") \
    .agg(_sum("total_amount").alias("total_spent"))

# Join with customer details
top_spenders = customer_spending.join(customer_df, on="customer_id", how="inner") \
    .orderBy(col("total_spent").desc()) \
    .limit(5)

top_spenders.select("customer_id", "customer_name", "email", "total_spent").show()


+-----------+-------------+------------------+-----------+
|customer_id|customer_name|             email|total_spent|
+-----------+-------------+------------------+-----------+
|        505|         Emma|  emma@example.com|      450.0|
|        556|       David2|david2@example.com|      400.0|
|        510|         Jack|  jack@example.com|      300.0|
|        520|          Tom|   tom@example.com|      280.0|
|        526|         Zach|  zach@example.com|      264.0|
+-----------+-------------+------------------+-----------+



In [79]:
#SALES
#1. Join Sales with Product Information (Inner Join)
sales_product_df = sales_df.join(product_df, on="product_id", how="inner")
sales_product_df.select("sale_id", "product_name", "category", "quantity", "price").show(5)


+-------+------------+-------------+--------+-----+
|sale_id|product_name|     category|quantity|price|
+-------+------------+-------------+--------+-----+
|   1001|    Widget A|      Gadgets|       3| 25.5|
|   1002|    Widget B|      Gadgets|       2| 15.0|
|   1003|    Widget C|  Electronics|       1| 30.0|
|   1004|    Widget A|      Gadgets|       5| 25.5|
|   1005|    Widget E|Home & Living|      10| 45.0|
+-------+------------+-------------+--------+-----+
only showing top 5 rows



In [81]:
#2. Join Sales with Customer Information (Inner Join)
sales_customer_df = sales_df.join(customer_df, on="customer_id", how="inner")
sales_customer_df.select("sale_id", "customer_name", "email", "sale_date", "quantity", "price").show(5)


+-------+-------------+-------------------+-------------------+--------+-----+
|sale_id|customer_name|              email|          sale_date|quantity|price|
+-------+-------------+-------------------+-------------------+--------+-----+
|   1001|        Alice|  alice@example.com|2025-07-10 08:23:00|       3| 25.5|
|   1002|          Bob|    bob@example.com|2025-07-11 09:45:00|       2| 15.0|
|   1003|      Charlie|charlie@example.com|2025-07-12 10:15:00|       1| 30.0|
|   1004|        David|  david@example.com|2025-07-13 12:20:00|       5| 25.5|
|   1005|         Emma|   emma@example.com|2025-07-14 14:35:00|      10| 45.0|
+-------+-------------+-------------------+-------------------+--------+-----+
only showing top 5 rows



In [83]:
#3. Inner Join with Filter on Category (e.g., Gadgets)
sales_gadgets_df = sales_df.join(product_df, on="product_id", how="inner") \
    .filter(product_df.category == "Gadgets")

sales_gadgets_df.select("sale_id", "product_name", "category", "quantity").show(5)


+-------+------------+--------+--------+
|sale_id|product_name|category|quantity|
+-------+------------+--------+--------+
|   1051|    Widget A| Gadgets|       3|
|   1004|    Widget A| Gadgets|       5|
|   1001|    Widget A| Gadgets|       3|
|   1052|    Widget B| Gadgets|       4|
|   1006|    Widget B| Gadgets|       4|
+-------+------------+--------+--------+
only showing top 5 rows



In [85]:
#4. Left Join: All Sales Even Without Product Info
left_join_df = sales_df.join(product_df, on="product_id", how="left")

left_join_df.select("sale_id", "product_id", "product_name", "category", "quantity").show(5)


+-------+----------+------------+-------------+--------+
|sale_id|product_id|product_name|     category|quantity|
+-------+----------+------------+-------------+--------+
|   1001|       101|    Widget A|      Gadgets|       3|
|   1002|       102|    Widget B|      Gadgets|       2|
|   1003|       103|    Widget C|  Electronics|       1|
|   1004|       101|    Widget A|      Gadgets|       5|
|   1005|       105|    Widget E|Home & Living|      10|
+-------+----------+------------+-------------+--------+
only showing top 5 rows



In [89]:
#5. Self-Join on sales_data (e.g., Compare Product Sales Across Dates)
from pyspark.sql.functions import col

sales_alias1 = sales_df.alias("s1")
sales_alias2 = sales_df.alias("s2")

self_join_df = sales_alias1.join(
    sales_alias2,
    (col("s1.product_id") == col("s2.product_id")) &
    (col("s1.sale_date") < col("s2.sale_date"))
).select(
    col("s1.sale_id").alias("sale_id_1"),
    col("s1.product_id").alias("product_id"),
    col("s1.sale_date").alias("sale_date_1"),
    col("s2.sale_id").alias("sale_id_2"),
    col("s2.sale_date").alias("sale_date_2")
)

self_join_df.show()



+---------+----------+-------------------+---------+-------------------+
|sale_id_1|product_id|        sale_date_1|sale_id_2|        sale_date_2|
+---------+----------+-------------------+---------+-------------------+
|     1001|       101|2025-07-10 08:23:00|     1051|2025-08-29 17:55:00|
|     1001|       101|2025-07-10 08:23:00|     1004|2025-07-13 12:20:00|
|     1002|       102|2025-07-11 09:45:00|     1052|2025-08-30 18:45:00|
|     1002|       102|2025-07-11 09:45:00|     1006|2025-07-15 16:00:00|
|     1003|       103|2025-07-12 10:15:00|     1053|2025-08-31 19:30:00|
|     1004|       101|2025-07-13 12:20:00|     1051|2025-08-29 17:55:00|
|     1005|       105|2025-07-14 14:35:00|     1055|2025-09-02 21:10:00|
|     1006|       102|2025-07-15 16:00:00|     1052|2025-08-30 18:45:00|
|     1007|       106|2025-07-16 17:10:00|     1056|2025-09-03 22:30:00|
|     1008|       107|2025-07-17 18:25:00|     1057|2025-09-04 23:50:00|
|     1009|       108|2025-07-18 19:30:00|     1058

In [91]:
#7. Join All Three: Sales + Product + Customer
full_sales_df = sales_df \
    .join(product_df, on="product_id", how="inner") \
    .join(customer_df, on="customer_id", how="inner")

full_sales_df.select("sale_id", "customer_name", "product_name", "category", "sale_date", "quantity", "price").show(5)


+-------+-------------+------------+-------------+-------------------+--------+-----+
|sale_id|customer_name|product_name|     category|          sale_date|quantity|price|
+-------+-------------+------------+-------------+-------------------+--------+-----+
|   1001|        Alice|    Widget A|      Gadgets|2025-07-10 08:23:00|       3| 25.5|
|   1002|          Bob|    Widget B|      Gadgets|2025-07-11 09:45:00|       2| 15.0|
|   1003|      Charlie|    Widget C|  Electronics|2025-07-12 10:15:00|       1| 30.0|
|   1004|        David|    Widget A|      Gadgets|2025-07-13 12:20:00|       5| 25.5|
|   1005|         Emma|    Widget E|Home & Living|2025-07-14 14:35:00|      10| 45.0|
+-------+-------------+------------+-------------+-------------------+--------+-----+
only showing top 5 rows



In [93]:
#3. Additional Transformations:
# Step 1: Filter Sales Between 2025-07-10 and 2025-07-15
from pyspark.sql.functions import col

filtered_sales_df = sales_df.filter(
    (col("sale_date") >= "2025-07-10") &
    (col("sale_date") <= "2025-07-15")
)

filtered_sales_df.show(truncate=False)



+-------+----------+-----------+-------------------+--------+-----+------------+
|sale_id|product_id|customer_id|sale_date          |quantity|price|total_amount|
+-------+----------+-----------+-------------------+--------+-----+------------+
|1001   |101       |501        |2025-07-10 08:23:00|3       |25.5 |76.5        |
|1002   |102       |502        |2025-07-11 09:45:00|2       |15.0 |30.0        |
|1003   |103       |503        |2025-07-12 10:15:00|1       |30.0 |30.0        |
|1004   |101       |504        |2025-07-13 12:20:00|5       |25.5 |127.5       |
|1005   |105       |505        |2025-07-14 14:35:00|10      |45.0 |450.0       |
+-------+----------+-----------+-------------------+--------+-----+------------+



In [95]:
#Step 2: Top 5 Customers by Total Spend
from pyspark.sql.functions import sum as _sum, col, desc

# Step 1: Calculate total_spent per sale
sales_with_total = sales_df.withColumn("total_spent", col("quantity") * col("price"))

# Step 2: Join with customer info
sales_customer = sales_with_total.join(customer_df, on="customer_id", how="inner")

# Step 3: Group by customer and sum total_spent
customer_spend = sales_customer.groupBy("customer_id", "customer_name").agg(
    _sum("total_spent").alias("total_spent")
)

# Step 4: Get Top 5 customers
top5_customers = customer_spend.orderBy(desc("total_spent")).limit(5)

top5_customers.show(truncate=False)


+-----------+-------------+-----------+
|customer_id|customer_name|total_spent|
+-----------+-------------+-----------+
|505        |Emma         |450.0      |
|556        |David2       |400.0      |
|510        |Jack         |300.0      |
|520        |Tom          |280.0      |
|526        |Zach         |264.0      |
+-----------+-------------+-----------+



In [97]:
# Step 3: Categorizing Customers Based on Spend
from pyspark.sql.functions import when

# Reuse the customer_spend DataFrame from previous step
categorized_customers = customer_spend.withColumn(
    "spend_category",
    when(col("total_spent") < 500, "Low")
    .when((col("total_spent") >= 500) & (col("total_spent") < 1500), "Medium")
    .otherwise("High")
)

categorized_customers.show(truncate=False)


+-----------+-------------+-----------+--------------+
|customer_id|customer_name|total_spent|spend_category|
+-----------+-------------+-----------+--------------+
|550        |Xena         |50.0       |Low           |
|501        |Alice        |76.5       |Low           |
|523        |Wendy        |100.0      |Low           |
|532        |Freddy       |236.0      |Low           |
|539        |Monica       |210.0      |Low           |
|540        |Nash         |55.5       |Low           |
|519        |Sara         |111.0      |Low           |
|502        |Bob          |30.0       |Low           |
|526        |Zach         |264.0      |Low           |
|528        |Bella        |70.0       |Low           |
|554        |Bob2         |200.0      |Low           |
|546        |Theo         |90.0       |Low           |
|516        |Paul         |128.0      |Low           |
|545        |Sophia       |91.5       |Low           |
|544        |Rob          |84.0       |Low           |
|534      

In [99]:
#Step 4: Finding the First and Last Purchase Date per Customer
from pyspark.sql.functions import min, max

# Get first and last purchase date for each customer
purchase_range_df = sales_df.groupBy("customer_id").agg(
    min("sale_date").alias("first_purchase"),
    max("sale_date").alias("last_purchase")
)

purchase_range_df.show(truncate=False)



+-----------+-------------------+-------------------+
|customer_id|first_purchase     |last_purchase      |
+-----------+-------------------+-------------------+
|540        |2025-08-18 21:25:00|2025-08-18 21:25:00|
|516        |2025-07-25 11:45:00|2025-07-25 11:45:00|
|513        |2025-07-22 23:40:00|2025-07-22 23:40:00|
|530        |2025-08-08 11:00:00|2025-08-08 11:00:00|
|501        |2025-07-10 08:23:00|2025-07-10 08:23:00|
|548        |2025-08-26 14:25:00|2025-08-26 14:25:00|
|539        |2025-08-17 20:15:00|2025-08-17 20:15:00|
|519        |2025-07-28 15:30:00|2025-07-28 15:30:00|
|556        |2025-09-03 22:30:00|2025-09-03 22:30:00|
|512        |2025-07-21 22:30:00|2025-07-21 22:30:00|
|570        |2025-09-17 23:00:00|2025-09-17 23:00:00|
|559        |2025-09-06 10:40:00|2025-09-06 10:40:00|
|545        |2025-08-23 11:40:00|2025-08-23 11:40:00|
|507        |2025-07-16 17:10:00|2025-07-16 17:10:00|
|544        |2025-08-22 10:30:00|2025-08-22 10:30:00|
|542        |2025-08-20 23:1

In [100]:
#Step 5: Customer Churn – Customers Who Haven’t Purchased in the Last 30 Days

from pyspark.sql.functions import col, to_date, lit
from datetime import datetime, timedelta

# Step 1: Define the cutoff date
today = datetime.strptime("2025-07-28", "%Y-%m-%d")
cutoff_date = (today - timedelta(days=30)).strftime("%Y-%m-%d")

# Step 2: Reuse the last_purchase dataframe if needed
# If not available, regenerate:
last_purchase_df = sales_df.groupBy("customer_id").agg(
    max("sale_date").alias("last_purchase")
)

# Step 3: Filter customers who haven't purchased in the last 30 days
churned_customers_df = last_purchase_df.filter(
    col("last_purchase") < lit(cutoff_date)
)

churned_customers_df.show(truncate=False)


+-----------+-------------+
|customer_id|last_purchase|
+-----------+-------------+
+-----------+-------------+

